<a href="https://colab.research.google.com/github/Ha-eunYu/Section_Project/blob/main/Section2/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = 'http://apis.data.go.kr/1480523/WaterQualityService/getSgisDrinkWaterList'
params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'XML', 'yyyy' : '2012,2013', 'sido' : '강원', 'period' : '2/4', 'sgg' : '강릉', 'legacyCodeNo' : 'PUB_2337,PUB_2340', 'spotNm' : '태장봉샘터' }

response = requests.get(url, params=params)
print(response.content)

b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>\n\t\t<returnReasonCode>30</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer 

cancer_dataset = load_breast_cancer()

features = cancer_dataset.data
target = cancer_dataset.target
df = pd.DataFrame(data=features, columns=cancer_dataset.feature_names)
df['target'] = target

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=2)
print(X_train.shape, X_test.shape)

(455, 30) (114, 30)


In [ ]:
import sys
import warnings

if "google.colab" in sys.modules:
    ## Colab에서 실행하는 경우 필요한 라이브러리를 미리 설치합니다.
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

warnings.filterwarnings("ignore")

In [ ]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [ ]:
from xgboost import XGBClassifier

pipe = make_pipeline(
    # OrdinalEncoder(),
    SimpleImputer(strategy='accuracy'),
    XGBClassifier(n_estimators=5
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2
                 )
)

pipe.fit(X_train, y_train);

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
scoring = 'accuracy'
n_iter = 5
random_state = 2
eli5 PermutationImportance 사용

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

rf = pipe.named_steps['randomforestclassifier']
importances = pd.Series(rf.feature_importances_, X_train.columns)

%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh();

# permuter 정의
permuter = PermutationImportance(
    pipe.named_steps['rf'], # model
    scoring='accuracy', # metric
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state=2
)

# permuter 계산은 preprocessing 된 X_val을 사용합니다.
X_val_transformed = pipe.named_steps['preprocessing'].transform(X_test)

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업입니다
permuter.fit(X_val_transformed, y_test);

KeyError: ignored

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = pipe.predict(X_test)
print('검증 정확도: ', accuracy_score(y_test, y_pred))

print(classification_report(y_pred, y_test))

검증 정확도:  0.956140350877193
              precision    recall  f1-score   support

           0       0.96      0.93      0.95        46
           1       0.96      0.97      0.96        68

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [ ]:
X_test

array([[1.394e+01, 1.317e+01, 9.031e+01, ..., 1.015e-01, 2.160e-01,
        7.253e-02],
       [1.496e+01, 1.910e+01, 9.703e+01, ..., 1.489e-01, 2.962e-01,
        8.472e-02],
       [9.668e+00, 1.810e+01, 6.106e+01, ..., 2.500e-02, 3.057e-01,
        7.875e-02],
       ...,
       [1.665e+01, 2.138e+01, 1.100e+02, ..., 2.095e-01, 3.613e-01,
        9.564e-02],
       [1.499e+01, 2.520e+01, 9.554e+01, ..., 2.899e-02, 1.565e-01,
        5.504e-02],
       [1.705e+01, 1.908e+01, 1.134e+02, ..., 2.543e-01, 3.109e-01,
        9.061e-02]])

In [ ]:
feature_names = X_test.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

AttributeError: ignored